**Мини-тест**: распознавание плана квартиры из изображения

**Цель**:
Показать, как вы решаете задачу извлечения архитектурной геометрии из изображения и превращаете её в пригодный для 2D/3D формат.

**Что нужно сделать**:
Собрать небольшой прототип, который принимает 3–5 изображений планов (JPG/PNG) и возвращает JSON с базовой структурой.

Можно использовать любой подход:

классический CV (OpenCV),

нейросети (YOLO/UNet/DeepLab/SAM и т.п.),

гибридный пайплайн.

Просьба кратко указать почему вы выбрали именно этот стек/модели для такого класса данных.

Минимальный ожидаемый результат

Сфокусироваться на стенах.

На выходе хотим получить JSON вида:

{
  "meta": { "source": "test_01.png" },
  "walls": [
    { "id": "w1", "points": [[x1,y1],[x2,y2]] },
    { "id": "w2", "points": [[x1,y1],[x2,y2],[x3,y3]] }
  ]
}


**Допустимо**:
- стены как линии или полилинии,
- координаты в пикселях,



Плюсом будет (если успеете)
Любой один пункт на выбор:
- двери/окна как простая детекция,
- контуры помещений (полигоны),
базовый OCR размеров (хотя бы демонстрация на 1–2 примерах),
простая обработка “грязного” фото (перспектива/шум).

Что прислать
репозиторий или архив с кодом;
короткий README:
- этапы пайплайна,
- какие модели/инструменты использованы и почему,
- где слабые места,
- что бы улучшили в следующей итерации.

Если вам удобнее оформить это как небольшой прикладной модуль с оценкой трудозатрат и ожидаемого результата — такой формат тоже подойдёт: нам важно увидеть ваш реальный подход и качество инженерного мышления.


Пусть есть изображение
![](room_1.jpg)

In [1]:
import cv2
import numpy as np

def detect_room(image_path):
    # Загрузка изображения и предобработка
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: не удалось загрузить изображение по пути {image_path}")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #blurred = cv2.GaussianBlur(gray, (5, 5), 0) # размытие не в этот раз
    #edged = cv2.Canny(blurred, 50, 150)         # получим только удвоение кол-ва контуров
    
    # Применить бинарный пороговый режим (объекты должны быть белыми на чёрном фоне).
    _, thresh = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY)

    # Поиск контуров (OpenCV 4.x)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    print(f"Найдено контуров: {len(contours)}")
    
    # Сортируем контуры по площади в порядке убывания
    # cv2.contourArea(contour) вычисляет площадь каждого контура
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Берем первые три самых больших контура
    # largest_contours = sorted_contours[:3]
    
    # Определяем цвета в формате BGR (Blue, Green, Red), который использует OpenCV
    #colors = [
    #    (0, 0, 255),  # Красный
    #    (255, 0, 0),  # Синий
    #    (0, 255, 0)   # Зеленый
    #]
    
    # 4. Рисуем контуры на изображении
    # Проходим по трем самым большим контурам и соответствующим им цветам
    #for i, contour in enumerate(largest_contours):
        # cv2.drawContours(изображение, [контур], -1, цвет, толщина)
        # -1 означает, что нужно нарисовать все контуры в списке (у нас он один)
        # толщина -1 означает, что контур будет закрашен (заполнен)
    #    cv2.drawContours(image, [contour], -1, colors[i], -1)

    if len(contours) >= 2:
        # Первый контур у нас рамка по ГОСТ
        # Сортируем и берем второй по величине контур
        sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
        second_largest_contour = sorted_contours[1]
    
        # Создаем маску и рисуем на ней контур
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [second_largest_contour], -1, 255, -1)
    
        # Создаем белый фон и копируем на него нужную область
        result_image = np.ones_like(image, dtype=np.uint8) * 255
        #image.copyTo(result_image, mask)
        result_image[mask > 0] = image[mask > 0]
    
        # Показываем исходное изображение и результат
        #cv2.imshow('Original Image', image)
        #cv2.imshow('Result - Second Largest Contour', result_image)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
    
    else:
        print("Найдено меньше двух контуров.")

    output_path = 'room_1_main.jpg'
    cv2.imwrite(output_path, result_image)
    print(f"\nРезультат сохранен в файл: {output_path}")

detect_room('room_1.jpg')

Найдено контуров: 1895

Результат сохранен в файл: room_1_main.jpg


Получили изображение с геометрией квартиры
![](room_1_main.jpg)

In [5]:
import cv2
import numpy as np

def detect_whalls(image_path, min_length=7):
    # Загрузка изображения и предобработка
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: не удалось загрузить изображение по пути {image_path}")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Применить бинарный пороговый режим.
    _, thresh = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY)
    
    # Создаем копию для изменений
    result_img = thresh.copy()
    height, width = result_img.shape

    print(f"Начало обработки изображения размером {width}x{height}...")

    # Проход по каждой строке
    for y in range(height):
        x = 0
        # Проход по пикселям в строке
        while x < width:
            # Находим начало черной последовательности
            if result_img[y, x] == 0: # Если пиксель черный
                start_x = x
                
                # Движемся до конца последовательности
                while x < width and result_img[y, x] == 0:
                    x += 1
                
                end_x = x # x теперь указывает на первый белый пиксель после последовательности (или на конец строки)
                
                # Вычисляем длину
                line_length = end_x - start_x
                
                # Проверяем условие и закрашиваем
                if 0 < line_length < min_length:
                    # Закрашиваем всю найденную короткую линию белым
                    result_img[y, start_x:end_x] = 255
            
            # Если пиксель был белый, просто переходим к следующему
            else:
                x += 1

    # Проход по каждому столбцу
    for x in range(width):
        y = 0
        # Проход по пикселям в столбце
        while y < height:
            # Находим начало черной последовательности
            if result_img[y, x] == 0: # Если пиксель черный
                start_y = y
                
                # Движемся до конца последовательности
                while y < height and result_img[y, x] == 0:
                    y += 1
                
                end_y = y # y теперь указывает на первый белый пиксель после последовательности (или на конец столбца)
                
                # Вычисляем длину
                line_length = end_y - start_y
                
                # Проверяем условие и закрашиваем
                if 0 < line_length < min_length:
                    # Закрашиваем всю найденную короткую линию белым
                    result_img[start_y:end_y, x] = 255
            
            # Если пиксель был белый, просто переходим к следующему
            else:
                y += 1
                
    print("Обработка завершена.")
    
    cv2.imwrite('room_1_main_whalls.jpg', result_img)

detect_whalls('room_1_main.jpg')

Начало обработки изображения размером 1518x1080...
Обработка завершена.


Получили стены
![](room_1_main_whalls.jpg)

In [3]:
import cv2
import numpy as np
import json
import math
import os


def trace_shape_by_algorithm(segments):
    """
    Обходит фигуру, следуя правилам, и возвращает список точек
    со стандартными типами данных Python (int).
    """
    # Преобразуем все координаты в стандартные int на входе функции
    clean_segments = []
    for p1, p2 in segments:
        p1_clean = (int(p1[0]), int(p1[1]))
        p2_clean = (int(p2[0]), int(p2[1]))
        clean_segments.append((p1_clean, p2_clean))

    # Работаем только с "чистыми" отрезками
    all_points = [p for seg in clean_segments for p in seg]
    start_point = min(all_points, key=lambda p: (p[1], p[0]))

    point_map = {}
    for p1, p2 in clean_segments:
        point_map.setdefault(p1, []).append((p1, p2))
        point_map.setdefault(p2, []).append((p1, p2))

    ordered_points = [start_point]
    current_point = start_point
    
    seg1, seg2 = point_map[start_point]
    
    pA = seg1[0] if seg1[1] == start_point else seg1[1]
    pB = seg2[0] if seg2[1] == start_point else seg2[1]

    if pA[0] > pB[0]:
        next_point = pA
    elif pB[0] > pA[0]:
        next_point = pB
    else:
        next_point = pA if pA[1] < pB[1] else pB

    ordered_points.append(next_point)
    previous_point = start_point
    current_point = next_point

    while True:
        connected_segs = point_map[current_point]
        
        next_seg = None
        for seg in connected_segs:
            if previous_point not in seg:
                next_seg = seg
                break
        
        p1, p2 = next_seg
        next_point = p1 if p2 == current_point else p2

        if next_point == start_point:
            break
            
        ordered_points.append(next_point)
        
        previous_point, current_point = current_point, next_point

    return ordered_points

def process_image_and_get_walls(image_path):
    """Основная функция для обработки изображения и извлечения данных о стенах."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: не удалось загрузить изображение {image_path}")
        return None, None # Возвращаем None для изображения и данных

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    walls_data = []
    for i, contour in enumerate(contours):
        epsilon = 0.001 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        #points = approx.reshape(-1, 2).tolist()
        vertices = [tuple(pt[0]) for pt in approx]
        
        if len(vertices) < 3:
            continue

        # Создаем список отрезков из вершин
        segments = []
        for j in range(len(vertices)):
            p1 = vertices[j]
            p2 = vertices[(j + 1) % len(vertices)]
            segments.append((p1, p2))

        # Обход контура по вашему алгоритму
        ordered_points = trace_shape_by_algorithm(segments)
        
        walls_data.append({
            "id": f"w{i+1}",
            "points": ordered_points
        })
    
    result_json = {
        "meta": { "source": os.path.basename(image_path) },
        "walls": walls_data
    }
    return image, result_json

# --- Новая функция для отрисовки ---

def draw_walls_on_image(image, walls_data, color=(0, 255, 0), thickness=2):
    """
    Рисует линии на изображении на основе данных о стенах.
    
    :param image: Исходное изображение (на котором будем рисовать).
    :param walls_data: Словарь с данными о стенах.
    :param color: Цвет линий в формате BGR (по умолчанию зеленый).
    :param thickness: Толщина линий.
    :return: Изображение с нарисованными линиями.
    """
    # Создаем копию изображения, чтобы не изменять оригинал
    image_with_lines = image.copy()

    if not walls_data or "walls" not in walls_data:
        print("Нет данных для отрисовки.")
        return image_with_lines

    for wall in walls_data["walls"]:
        # Извлекаем точки
        points = wall["points"]
        
        # cv2.polylines требует, чтобы точки были в формате numpy массива
        # и обернуты в дополнительный список, даже если мы рисуем один полигон.
        pts_array = np.array(points, dtype=np.int32)
        
        # Рисуем полигон
        # isClosed=True соединит последнюю точку с первой
        cv2.polylines(image_with_lines, [pts_array], isClosed=True, color=color, thickness=thickness)
        
    return image_with_lines

# --- Пример использования ---

# Создаем тестовое изображение
def create_test_image(filename="test_room.png", width=640, height=480):
    img = np.ones((500, 700, 3), dtype=np.uint8) * 255
    """Создает тестовое изображение с различными фигурами."""
    img = np.zeros((height, width, 3), dtype=np.uint8)
    img.fill(255) # Белый фон

    # Фигура 1: Прямоугольник
    cv2.rectangle(img, (50, 50), (200, 150), (0, 0, 0), -1)

    # Фигура 2: L-образная фигура
    pts_l = np.array([[250, 50], [350, 50], [350, 100], [300, 100], [300, 150], [250, 150]], np.int32)
    cv2.fillPoly(img, [pts_l], (0, 0, 0))

    # Фигура 3: Большой квадрат
    cv2.rectangle(img, (400, 250), (550, 400), (0, 0, 0), -1)

    # Фигура 4 (шум): Треугольник - должен быть отфильтрован
    pts_tri = np.array([[50, 250], [100, 350], [20, 350]], np.int32)
    cv2.fillPoly(img, [pts_tri], (0, 0, 0))

    # Фигура 5 (шум): Круг - должен быть отфильтрован
    cv2.circle(img, (450, 100), 40, (0, 0, 0), -1)
    cv2.imwrite(filename, img)
    return filename

#image_path = 'test_1_main_whalls.jpg'
#image_file = create_test_image(image_path)

image_path = 'room_1_main_whalls.jpg'

# Обрабатываем изображение и получаем данные
original_image, walls_data = process_image_and_get_walls(image_path)

# Если данные успешно получены, рисуем линии
if original_image is not None and walls_data is not None:
    # Выводим JSON для информации
    print("--- Данные о стенах ---")
    print(json.dumps(walls_data, indent=4, ensure_ascii=False))
    
    # Рисуем стены на изображении
    # Можно изменить цвет, например, на красный (0, 0, 255) или синий (255, 0, 0)
    result_image = draw_walls_on_image(original_image, walls_data, color=(0, 0, 255), thickness=3)
    cv2.imwrite('room_1_main_polylines.jpg', result_image)

    # Показываем исходное изображение и результат
    # cv2.imshow('Original Image', original_image)
    # cv2.imshow('Image with Detected Walls', result_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

--- Данные о стенах ---
{
    "meta": {
        "source": "room_1_main_whalls.jpg"
    },
    "walls": [
        {
            "id": "w1",
            "points": [
                [
                    337,
                    633
                ],
                [
                    376,
                    633
                ],
                [
                    377,
                    738
                ],
                [
                    759,
                    738
                ],
                [
                    760,
                    647
                ],
                [
                    832,
                    647
                ],
                [
                    832,
                    663
                ],
                [
                    777,
                    664
                ],
                [
                    777,
                    754
                ],
                [
                    337,
                    

На изображении отметим полилинии с геометрией квартиры
![](room_1_main_polylines.jpg)
Всё корректно.

In [4]:
import cv2
import numpy as np
from scipy.spatial import KDTree # Импортируем KD-Tree

def keep_dominant(input_path, output_path):
    """
    Оставляет на изображении только пиксели, в которых синий канал преобладает
    над красным и зеленым. Остальные пиксели заменяются на белые.

    Args:
        input_path (str): Путь к исходному изображению.
        output_path (str): Путь для сохранения обработанного изображения.
    """
    # Загрузка изображения в цвете (BGR)
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Ошибка: Не удалось загрузить изображение по пути '{input_path}'")
        return

    print(f"Изображение '{input_path}' успешно загружено. Размер: {img.shape}")

    # Разделение изображения на цветовые каналы
    # OpenCV работает с форматом BGR (Blue, Green, Red)
    b, g, r = cv2.split(img)

    # Создание маски для пикселей с преобладанием синего
    # Условие: синий (b) должен быть строго больше красного (r) И зеленого (g)
    # Результатом будет двумерный массив из True и False
    blue_mask = (b > r+5) & (b > g+5)

    # Создание белого фона и применение маски
    # Создаем новое изображение, заполненное белым цветом
    white_img = np.full_like(img, 255)

    # Копируем пиксели с исходного изображения на белый фон,
    # но только те места, где маска равна True
    white_img[blue_mask] = img[blue_mask]

    # 5. Сохранение результата
    cv2.imwrite(output_path, white_img)
    print(f"Обработка завершена. Результат сохранен в файл '{output_path}'")

def draw_blue_dots_on_non_white(input_path, output_path, neighbor_radius=20):
    """
    Находит все небелые пиксели и рисует синюю точку только в тех из них,
    у которых есть хотя бы один другой небелый пиксель в указанном радиусе.

    Args:
        input_path (str): Путь к исходному изображению.
        output_path (str): Путь для сохранения обработанного изображения.
        neighbor_radius (int): Радиус поиска соседей в пикселях.
    """
    # Загрузка изображения
    img = cv2.imread(input_path)
    if img is None:
        print(f"Ошибка: Не удалось загрузить изображение по пути '{input_path}'")
        return

    print(f"Изображение '{input_path}' успешно загружено. Размер: {img.shape}")

    # Создание маски и получение координат всех небелых пикселей
    non_white_mask = np.sum(img, axis=2) < 750
    y_coords, x_coords = np.where(non_white_mask)

    # Преобразуем координаты в формат (N, 2), где N - число точек
    # KD-Tree ожидает массив точек вида [[x1, y1], [x2, y2], ...]
    points = np.column_stack((x_coords, y_coords))

    if len(points) == 0:
        print("На изображении не найдено небелых пикселей.")
        return

    print(f"Найдено {len(points)} небелых пикселей-кандидатов.")

    # Построение KD-Tree для эффективного поиска соседей
    # Это позволяет очень быстро находить все точки в заданном радиусе
    tree = KDTree(points)

    # Поиск соседей для каждой точки
    # query_ball_point возвращает для каждой точки список индексов ее соседей (включая саму себя)
    neighbors_list = tree.query_ball_point(points, r=neighbor_radius)

    # Фильтрация точек
    # Оставляем только те точки, у которых есть соседи (т.е. есть хотя бы один другой)
    valid_indices = [i for i, neighbors in enumerate(neighbors_list) if len(neighbors) > 5]
    
    if not valid_indices:
        print("Не найдено точек, удовлетворяющих условию соседства.")
        # Сохраняем исходное изображение, если ничего не найдено
        cv2.imwrite(output_path, img)
        print(f"Исходное изображение сохранено в '{output_path}'")
        return

    valid_points = points[valid_indices]
    print(f"Из них отобрано {len(valid_points)} точек для отрисовки.")

    # Рисование синих точек на отфильтрованных координатах
    dot_radius = 3
    dot_color = (255, 0, 0)  # Синий в BGR

    for x, y in valid_points:
        cv2.circle(img, (x, y), dot_radius, dot_color, -1)

    # 7. Сохранение результата
    cv2.imwrite(output_path, img)
    print(f"Обработка завершена. Результат сохранен в файл '{output_path}'")


# --- Основной блок выполнения ---
if __name__ == "__main__":
    # Укажите имя вашего исходного файла здесь
    input_filename = "room_1_main.jpg"
    # Имя выходного файла
    output_filename1 = "windows1.jpg"
    output_filename2 = "windows2.jpg"

    # Вызываем функцию для обработки
    keep_dominant(input_filename, output_filename1)
    draw_blue_dots_on_non_white(output_filename1, output_filename2)


Изображение 'room_1_main.jpg' успешно загружено. Размер: (1080, 1518, 3)
Обработка завершена. Результат сохранен в файл 'windows1.jpg'
Изображение 'windows1.jpg' успешно загружено. Размер: (1080, 1518, 3)
Найдено 3069 небелых пикселей-кандидатов.
Из них отобрано 3061 точек для отрисовки.
Обработка завершена. Результат сохранен в файл 'windows2.jpg'


Расположение окон
![](windows2.jpg)

Это простой пример реального подхода и качества инженерного мышления. 

**Фотограмметрия** - научно-техническая дисциплина, занимающаяся определением формы, размеров, положения и иных характеристик объектов по их изображениям.
    
Финансирование желательно улучшить в следующей итерации.

https://github.com/AlexandrParkhomenko/cv/blob/main/room.ipynb